In [1]:
import tensorflow as tf
import numpy as np
import random
import os
import glob
from keras.preprocessing.image import img_to_array, load_img
from PIL import Image

In [2]:
dict_labels = {"daisy":0, "dandelion":1, "rose":2, "sunflower":3, "tulip":4}
size = (224, 224)

In [3]:
for folders in glob.glob("/Users/username/Documents/data/train/*"):
    print(folders)
    images = []
    labels_hot = []
    labels = []
     
    for filename in os.listdir(folders):
        label = os.path.basename(folders)
        className = np.asarray(label)
        
        img = load_img(os.path.join(folders,filename))
        img = img.resize(size,Image.BILINEAR)

        if img is not None:
            if label is not None:
                labels.append(className)
                labels_hot.append(dict_labels[label])
            x=img_to_array(img)
            images.append(x)
        
    images = np.array(images)/255.    
    labels_hot = np.array(labels_hot)
    print("images.shape={}, labels_hot.shape=={}".format(images.shape, labels_hot.shape))    
    imagesavepath='train_dataset/'
    if not os.path.exists(imagesavepath):
        os.makedirs(imagesavepath)
    np.save(imagesavepath+'{}_images.npy'.format(label),images)    
    np.save(imagesavepath+'{}_label.npy'.format(label),labels)    
    np.save(imagesavepath+'{}_labels_hot.npy'.format(label),labels_hot)
    print('{} files has been saved.'.format(label))

In [4]:
import csv
mapping_table={}
with open('mapping_table.csv', 'r', encoding="utf8", errors='ignore', newline = '') as csvname:
    rows = csv.reader(csvname)

    for row in rows:
        mapping_table[row[0]]=row[1]

In [5]:
folder = "/Users/username/Documents/data/test/"
images = []
    
for n in range(1,434):
    img = load_img(os.path.join(folder,mapping_table[str(n)]))
    img = img.resize(size,Image.BILINEAR)

    x=img_to_array(img)
    images.append(x)

images = np.array(images)/255.    

print("images.shape={}".format(images.shape))    
imagesavepath='test_dataset/'
if not os.path.exists(imagesavepath):
    os.makedirs(imagesavepath)
np.save(imagesavepath+'test_images.npy',images)

In [6]:
folder = "/Users/username/Documents/train_dataset"
filename = ["dandelion", "rose", "sunflower", "tulip"]
train_images = np.load(os.path.join(folder,"daisy_images.npy"))
train_labels = np.load(os.path.join(folder,"daisy_labels_hot.npy"))
    
for name in filename:
    images = np.load(os.path.join(folder,name+"_images.npy"))
    train_images = np.append(train_images,images,axis=0)
    
    labels = np.load(os.path.join(folder,name+"_labels_hot.npy"))
    train_labels = np.append(train_labels,labels,axis=0)

In [7]:
from sklearn.model_selection import train_test_split
train_images, validation_images, train_labels, validation_labels = train_test_split(train_images, train_labels, test_size=0.2, shuffle=True)

In [8]:
import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam, SGD

In [9]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

net = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224,224,3))

In [10]:
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(5, activation='softmax', name='softmax')(x)

In [11]:
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers:
    layer.trainable = True

In [12]:
net_final.compile(optimizer=SGD(lr=5e-3),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [13]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rotation_range=50, width_shift_range=0.5, height_shift_range=0.5,
                             shear_range=0.5, zoom_range=0.5, channel_shift_range=0.5, horizontal_flip=True, fill_mode='nearest')

In [14]:
best_model_filepath = './best_model.h5'
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
saveBestModel = tf.keras.callbacks.ModelCheckpoint(best_model_filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')

In [15]:
batch_size=32
epochs=100

datagen.fit(train_images)
net_final.fit_generator(datagen.flow(train_images, train_labels, batch_size=batch_size),
                        steps_per_epoch=len(train_images)/batch_size, epochs=epochs, verbose=1, validation_data=(validation_images, validation_labels), callbacks=[earlyStopping, saveBestModel])

In [16]:
model = load_model('best_model.h5')

In [17]:
folder = "test_dataset"
test_images = np.load(os.path.join(folder,"test_images.npy"))

In [18]:
predictions = model.predict(test_images)

In [19]:
import csv

with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'Category'])
    
for p in range(0,433):
    row = []
    row.append(p+1)
    predict = predictions[p].tolist()
    row.append(predict.index(max(predict)))
    with open('output.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(row)